The file contains the edges of a directed graph. Vertices are labeled as positive integers from 1 to 875714. Every row indicates an edge, the vertex label in first column is the tail and the vertex label in second column is the head (recall the graph is directed, and the edges are directed from the first column vertex to the second column vertex). So for example, the 11th row looks liks : "2 47646". This just means that the vertex with label 2 has an outgoing edge to the vertex with label 47646

Your task is to code up the algorithm from the video lectures for computing strongly connected components (SCCs), and to run this algorithm on the given graph.

Output Format: You should output the sizes of the 5 largest SCCs in the given graph, in decreasing order of sizes, separated by commas (avoid any spaces). So if your algorithm computes the sizes of the five largest SCCs to be 500, 400, 300, 200 and 100, then your answer should be "500,400,300,200,100" (without the quotes). If your algorithm finds less than 5 SCCs, then write 0 for the remaining terms. Thus, if your algorithm computes only 3 SCCs whose sizes are 400, 300, and 100, then your answer should be "400,300,100,0,0" (without the quotes). (Note also that your answer should not have any spaces in it.)

WARNING: This is the most challenging programming assignment of the course. Because of the size of the graph you may have to manage memory carefully. The best way to do this depends on your programming language and environment, and we strongly suggest that you exchange tips for doing this on the discussion forums.

In [2]:
import os
filename = './_410e934e6553ac56409b2cb7096a44aa_SCC.txt'
edges = [(int(line.rstrip().split(' ')[0]),int(line.rstrip().split(' ')[1])) for line in open(filename)]

In [53]:
from collections import defaultdict
import sys
import threading
from itertools import groupby

In [8]:
sys.setrecursionlimit(2 ** 20)
threading.stack_size(67108864)

67108864

In [30]:
# create a graph adjacency dictionary
def create_graph(edges):
    d = defaultdict(list)
    for k,v in edges:
        d[k].append(v)
    return d
        
    

In [31]:
# original graph
o_graph = create_graph(edges)

In [39]:
# create a graph with arcs reversed
def reverse_graph(graph):
    d = defaultdict(list)
    for k,v in graph.items():
        for j in v:
            d[j].append(k)
    return d
    

In [40]:
# reverse graph
r_graph = reverse_graph(o_graph)

In [36]:
print len(r_graph),len(o_graph)

875714 739454


In [105]:
# create five global variables
# t is the # of nodes processed so far, for finishing times in 1st pass
t = 0
# s is the current source vertex, for leaders in 2nd pass
s = None
# leader node
leader = {}
# finish time
finish = {}
# visited nodes
visited = set()

In [62]:
# DFS loop algorithm
def dfs_loop(graph,nodes):
    global s,visited
    for i in nodes:
        if i not in visited:
            s = i
            dfs(graph,i)
    

In [63]:
# DFS function
def dfs(graph,i):
    visited.add(i)
    global leader,t,s
    leader[i] = s
    for j in graph[i]:
        if j not in visited:
            dfs(graph,j)
    t = t + 1
    finish[i] = t

In [97]:
# SCC computation
def scc(o_graph,r_graph):
    global t,s,leader,finish,visited
    # run dfs_loop on r_graph
    nodes = set()
    for k,v in r_graph.items():
        nodes = nodes | set(v)
        nodes.add(k)
    nodes = sorted(list(nodes),reverse = True)
    dfs_loop(r_graph,nodes)
    # record finish time
    sorted_nodes = sorted(finish,key = finish.get,reverse = True)
    # reset t,s,leader,finish
    t = 0
    s = None
    leader = {}
    finish = {}
    visited = set()
    # run dfs loop on o_graph
    dfs_loop(o_graph,sorted_nodes)
    # return leaders
    out = defaultdict(list)
    sorted_scc = sorted(leader,key = leader.get)
    for leader,nodes in groupby(sorted_scc,key = leader.get):
        out[leader] = list(nodes)
    # set t,s,leader,finish
    t = 0
    s = None
    leader = {}
    finish = {}
    visited = set()
    return out

In [ ]:
def largest_scc(scc):
    len_ls = []
    for i in scc.keys():
        len_ls.append(len(scc[i]))
    if len(len_ls) < 5:
        len_ls += [0] * (5 - len(len_ls))
    return sorted(len_ls,reverse = True)

In [64]:
sample = [[1,7],[4,1],[7,4],[7,9],[9,6],[6,3],[3,9],[6,8],[8,2],[2,5],[5,8]]

In [67]:
o_graph_s = create_graph(sample)
r_graph_s = reverse_graph(o_graph_s)

In [99]:
sample_scc = scc(o_graph_s,r_graph_s)

In [100]:
sample_scc

defaultdict(list, {7: [1, 4, 7], 8: [2, 5, 8], 9: [3, 6, 9]})

In [107]:
sample_2 = [[1,4],[2,8],[3,6],[4,7],[5,2],[6,9],[7,1],[8,5],[8,6],[9,7],[9,3]]
o_graph_2 = create_graph(sample_2)
r_graph_2 = reverse_graph(o_graph_2)
scc(o_graph_2,r_graph_2)

defaultdict(list, {7: [1, 4, 7], 8: [2, 5, 8], 9: [3, 6, 9]})

In [109]:
sample_3 = [[1,2],[2,3],[3,1],[3,4],[5,4],[6,4],[8,6],[6,7],[7,8]]
o_graph_3 = create_graph(sample_3)
r_graph_3 = reverse_graph(o_graph_3)
scc(o_graph_3,r_graph_3)

defaultdict(list, {3: [1, 2, 3], 4: [4], 5: [5], 8: [6, 7, 8]})

In [ ]:
# run scc on the programming assignment file
largest_scc(scc(o_graph,r_graph))[0:5]